# **Monitoring pasażerów za pomocą algorytmu SafeSO**

W niniejszym ćwiczeniu wykorzystamy podejście zaproponowane w ramach algorytmu SafeSO do monitorignu pasażerów na tylnym siedzeniu pojazdu. Zastosujemy przy tym sieć MobileNetV2 i porównamy jej wyniki na tle innych, przetestowanych przez autorów SafeSO.

# Przygotowanie bazy danych
W naszych badaniach użyjemy zbioru SVIRO. Zawiera on sztucznie wygenerowane obrazy tylnych siedzeń różnych pojazdów, które zostały zaklasyfikowane do 7 kategorii:
0. puste siedzenie,
1. niemowlę w foteliku,
2. dziecko w foteliku,
3. osoba dorosła,
4. rzecz codziennego użytku (np. poduszka, torba),
5. pusty fotelik niemowlęcy,
6. pusty fotelik dziecięcy.

W naszych badaniach, podobnie jak to było w przypadku oryginalnych prac nad algorytmem SafeSO, wykorzystamy kolorowe (RGB) obrazy wnętrzna samochodu BMW x5, z wyciętymi pojedynczymi siedzeniami.

Zbiór SVIRO jest ogólnie dostępny na dedykowanej mu [stronie internetowej](https://sviro.kl.dfki.de/). Jak już wspomniano, zawiera on ogólnie zdjęcia różnych pojazdów, dodatkowo w kilku wersjach (skala szarości, RGB, głębia). Interesujący nas wariant możemy pobrać i przygotować do wykorzystania za pomocą poniższych komend (proszę się upewnić, czy wszystkie z nich są zrozumiałe!).

**Uwaga.** W zależności od łącza internetowego, proces pobierania może potrwać nawet kilka minut.

In [ ]:
!wget https://sviro.kl.dfki.de/download/bmw-x5-4/?wpdmdl=423
!unzip index.html?wpdmdl=423
!rm index.html?wpdmdl=423

--2023-10-18 11:29:21--  https://sviro.kl.dfki.de/download/bmw-x5-4/?wpdmdl=423
Resolving sviro.kl.dfki.de (sviro.kl.dfki.de)... 193.175.65.22
Connecting to sviro.kl.dfki.de (sviro.kl.dfki.de)|193.175.65.22|:443... failed: Connection timed out.
Retrying.

--2023-10-18 11:31:31--  (try: 2)  https://sviro.kl.dfki.de/download/bmw-x5-4/?wpdmdl=423
Connecting to sviro.kl.dfki.de (sviro.kl.dfki.de)|193.175.65.22|:443... failed: Connection timed out.
Retrying.

--2023-10-18 11:33:43--  (try: 3)  https://sviro.kl.dfki.de/download/bmw-x5-4/?wpdmdl=423
Connecting to sviro.kl.dfki.de (sviro.kl.dfki.de)|193.175.65.22|:443... failed: Connection timed out.
Retrying.

--2023-10-18 11:35:55--  (try: 4)  https://sviro.kl.dfki.de/download/bmw-x5-4/?wpdmdl=423
Connecting to sviro.kl.dfki.de (sviro.kl.dfki.de)|193.175.65.22|:443... failed: Connection timed out.
Retrying.

--2023-10-18 11:38:11--  (try: 5)  https://sviro.kl.dfki.de/download/bmw-x5-4/?wpdmdl=423
Connecting to sviro.kl.dfki.de (sviro.kl.dfki

Teraz przystępujemy do realizacji właściwego zadania - najpierw importujemy wszystkie potrzebne pakiety:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import time
import copy
import numpy as np
from sklearn import metrics

Ustawiamy ścieżki do odpowiednich zbiorów (uczącego i testowego):

In [ ]:
SVIRO_train_path =  # FIXME - np. ścieżka: './x5/train/RGB'
SVIRO_test_path = # FIXME - np. ścieżka: './x5/test_with_labels/RGB'

Jak już wspomniano, w dalszej części ćwiczenia będziemy wykorzystywali sieć MobileNetV2. Na wejściu pierwszej warstwy wymaga ona tensorów w rozmiarze [3 x 224 x 224]. O ile pierwszy z wymienionych wymiarów jest zapewniony poprzez wykorzystanie obrazów RGB, o tyle pozostałe dwa musimy uzyskać za pomocą odpowiednich transformacji (oryginalne obrazy są bowiem prostokątami). Zastosujemy przy tym często spotykane podejście: najpierw przeskalujemy obraz w ten sposób, że krótszy bok będzie miał długość zbliżoną do docelowej (przyjmijmy 256), a następnie wytniemy z niego kwadrat o pożądanych rozmiarach.

**Zadanie.** Proszę uzupełnić zestaw przekształceń preprocessingu, wykorzystując moduł transforms z pakietu torchvision i metody: Resize, CenterCrop oraz ToTensor. Oczywiście kolejność przekształceń ma znaczenie. W razie wątpliwości można skorzystać z [dokumentacji modułu transforms](https://pytorch.org/vision/stable/transforms.html).

In [ ]:
preprocessing = transforms.Compose([]) # TODO - uzupełnić tablicę []

Pakiet torchvision udostępnia wygodne API do pobierania wielu popularnych zbiorów danych. Niestety, do tego grona nie zalicza się SVIRO. Możemy jednak wykorzystać ogólną klasę ImageFolder do obsłużenia naszego zbioru.

**Zadanie.** Po zapoznaniu się z [dokumentacją ImageFolder](https://pytorch.org/vision/stable/datasets.html#torchvision.datasets.ImageFolder) proszę wykorzystać tę klasę do obsłużenia uczącego i testowego zbioru SVIRO. W obu przypadkach wystarczą dwa argumenty: ścieżka i zestaw transformacji (taki sam dla obu zbiorów).

In [ ]:
SVIRO_train = # TODO - zbiór uczący
SVIRO_test = # TODO - zbiór testowy

Oprócz dwóch wspomnianych zbiorów, będziemy również potrzebowali zbioru walidacyjnego do kontrolowania procesu uczenia. Możemy go utworzyć poprzez losowy podział oryginalnego zbioru uczącego, np. 10% przeznaczyć na zbiór walidacyjny, a pozostałą część na właściwy zbiór uczący. Do tego celu służy metoda [SubsetRandomSampler](https://pytorch.org/docs/stable/data.html#torch.utils.data.SubsetRandomSampler), z której skorzystamy w naszym przypadku, a następnie wświetlimy finalną liczbę obrazów w każdym zbiorze:

In [ ]:
val_split = 0.1
SVIRO_train_size = len(SVIRO_train)
split = int(np.floor(val_split * SVIRO_train_size))

indices = list(range(SVIRO_train_size))
np.random.seed(13)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

dataset_sizes = {'train': len(train_indices), 'val': len(val_indices), 'test': SVIRO_test.__len__()}

for x in ['train', 'val', 'test']:
    print('[INFO] Number of ' + x + ' samples: ' + str(dataset_sizes[x]))

Oprócz pobierania zawartości danego zbioru (czy to z internetu, czy z dysku lokalnego), PyTorch umożliwia również wygodne wczytywanie poszczególnych obrazów za pomocą klasy DataLoader.

**Zadanie.** Po zapoznaniu się z [dokumentacją DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) proszę przygotować loadery dla każdego z trzech zbiorów: uczącego, walidacyjnego i testowego. We wszystkich przypadkach będziemy potrzebowali trzech argumentów: źródła danych (*dataset*) - SVIRO_train lub SVIRO_test, rozmiaru batcha (*batch_size*) - wszędzie jednakowy oraz *pin_memory* z wartością True. Ostatni argument umożliwia przyspieszenie wczytywania danych do GPU z wykorzystaniem CUDA. W przypadku loaderów zbioru uczącego i walidacyjnego musimy również wykorzystać argument *sampler* z przygotowanymi wcześniej obiektami klasy SubsetRandomSampler.

In [ ]:
batch_size = 64
train_loader = # TODO - loader dla zbioru uczącego
val_loader = # TODO - loader dla zbioru walidacyjnego
test_loader = # TODO - loader dla zbioru testowego

dataloaders = {'train': train_loader, 'val': val_loader, 'test': test_loader}

Naszą docelową platformę wykonywania obliczeń stanowi GPU z CUDA (o ile tylko jest dostępne), co możemy wyrazić przez instrukcję warunkową przedstawioną poniżej.

**Uwaga.** Proszę dobrze zapamiętać poniższą instrukcję, gdyż będzie ona nam wielokrotnie potrzebna.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Funkcja ucząca i model

Będziemy przeprowadzać dwukrotne uczenie sieci neuronowej, stąd warto zdefiniować odrębną funkcję uczącą.

**Zadanie.** Zdefiniować funkcję uczącą na podstawie [poradnika o Transfer Learningu w Pytorchu](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html#training-the-model).

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    # TODO
    return model

Pakiet torchvision udostępnia szereg modeli sieci neuronowych wytrenowanych na zbiorze ImageNet. Wśród nich znajduje się również model sieci MobileNetV2, który wczytujemy:

In [ ]:
model = models.mobilenet_v2(pretrained=True)

print(model)

# Transfer Learning

Wczytana przez nas sieć jest przystosowana do klasyfikacji obiektów (zwierząt) pochodzących z 1000 klas. Jak już wspomniano, w zbiorze SVIRO mamy jedynie 7 klas. Wobec tego musimy zmienić klasyfikator oraz przystosować całą sieć do nowego zbioru danych. Wzorując się na algorytmie SafeSO, dokonamy tego w dwóch etapach: najpierw nauczymy sam klasyfikator na podstawie cech wykrywanych przez oryginalną sieć, a następnie dokonany finetuningu całości. Przy wykonywaniu tych kroków można wzorować się na wspomnianym już [poradniku o Transfer Learningu w PyTorchu](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html).

## Uczenie klasyfikatora

W pierwszym etapie musimy przede wszystkim zmienić klasyfikator sieci i przeprowadzić jego uczenie. Zgodnie z przyjętą metodologią, potrzebujemy równocześnie "zamrozić" parametry warstw odpowiedzialnych za ekstrakcję cech. Najprościej zrealizować to w następującej kolejności:
1. Zamrozić **wszystkie** parametry oryginalnej sieci (doprowadzi to również do zamrożenia parametrów oryginalnego klasyfikatora, ale zaraz go i tak wymienimy, więc nie ma to dla nas znaczenia).
2. Wymienić klasyfikator na właściwy dla naszego problemu.
3. Przeprowadzić uczenie tak przygotowanej sieci.

**Zadanie.** Proszę zamrozić parametry oryginalnej sieci za pomocą odpowiedniego ustawienia flagi *requires_grad*.

In [ ]:
for param in model.parameters():
    # TODO

**Zadanie.** Proszę zdefiniować nowy klasyfikator dla naszej sieci. Ma on mieć taką samą strukturę i parametry, jak oryginalny klasyfikator (por. rezultat wywołania *print(model)* powyżej), tylko zamiast 1000 cech wyjściowych (*out_features*) powinien mieć ich 7 (tyle, ile klas w zbiorze danych). Należy skorzystać z warstw *nn.Dropout* oraz *nn.Linear*, a do ich połączenia można użyć kontenera *nn.Sequential*. Informacje o nich znajdują się w dokumentacji [torch.nn](https://pytorch.org/docs/stable/nn.html).

In [ ]:
new_cls = # TODO
model.classifier = new_cls

print(model)

Gotowy model umieszczamy w pamięci urządzenia, na którym pracujemy (np. GPU z CUDA):

In [ ]:
model.to(device)

Możemy teraz przystąpić do uczenia naszego modelu. W tym celu musimy najpierw zdefiniować funkcję błędu, optymalizator i scheduler. Co istotne, w pierwszym etapie chcemy uczyć jedynie parametry klasyfikatora naszego modelu (można je znaleźć poprzez wywołanie *model.classifier.parameters()*), więc to na nich powinien działać optymalizator.

**Zadanie.** Proszę zdefiniować *CrossEntropyLoss* jako funkcję błędu, optymalizator *Adam* (ze współczynnikiem uczenia *lr=1e-2*) oraz scheduler *StepLR* (współczynnik *gamma=0.1*, *step_size* można natomiast testować różny, a na początek przyjąć 3). Należy pamiętać o przekazaniu właściwych parametrów sieci do optymalizatora. W razie wątpliwości można wesprzeć się dokumentacją
modułów [torch.nn](https://pytorch.org/docs/stable/nn.html) i [torch.optim](https://pytorch.org/docs/stable/optim.html).

In [ ]:
criterion = # TODO - funkcja błędu
optimizer_cls = # TODO - optymalizator
scheduler_cls = # TODO - scheduler

Przeprowadzamy uczenie naszego modelu, wykorzystując zdefiniowaną uprzednio funkcję *train_model*.

**Uwaga.** Może to potrwać kilka minut.

In [ ]:
model = train_model(model=model, criterion=criterion, optimizer=optimizer_cls, scheduler=scheduler_cls, num_epochs=5)

Podobnie, jak uczenie, tak i ewaluację będziemy przeprowadzać dwukrotnie. W związku z tym warto przygotować sobie ogólną funkcję do tego celu.

**Zadanie.** Proszę zapoznać się z poniższą funkcją ewaluacyjną (np. czekając, aż sieć się nauczy). Szczególną uwagę należy zwrócić na instrukcje związane z obsługą gradientów oraz sposób konwersji wyników z PyTorcha do Numpy.

In [ ]:
def test_model(model):
    predictions = np.array([])
    true_values = np.array([])

    model.eval()

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        predictions = np.append(predictions, preds.detach().cpu().numpy())
        true_values = np.append(true_values, labels.data.numpy())

    return true_values, predictions

Na koniec tego etapu przeprowadzamy ewaluację uzyskanych wyników. Posłużymy się przy tym macierzą błędów (ang. *confusion matrix*) oraz wynikającymi z niej parametrami, obliczonymi za pomocą modułu *metrics* z pakietu *scikit-learn*.


In [ ]:
true_values, predictions = test_model(model)

print(metrics.classification_report(true_values, predictions, digits=3))

## Finetuning

Po wstępnym nauczeniu klasyfikatora, przechodzimy do drugiego etapu. Teraz traktujemy sieć jako całość zainicjalizowaną pretrenowanymi parametrami:
- warstwy odpowiedzialne za ekstrakcję cech mają wagi nauczone na zbiorze zwierząt,
- klasyfikator ma wagi wyuczone przed chwilą przez nas.

Tym razem sprawa jest prostsza: wystarczy odmrozić wszystkie wagi i można przystąpić do uczenia.

**Zadanie.** Proszę odmrozić uczenie wszystkich parametrów modelu poprzez odpowiednie ustawienie flagi *requires_grad*:

In [ ]:
for param in model.parameters():
    # TODO

**Zadanie.** Proszę zdefiniować nowy optymalizator (ponownie *Adam*, tylko z dostępem do **wszystkich** parametrów modelu oraz ze współczynnikiem uczenia *lr=1e-4*) oraz nowy scheduler (taki, jak poprzednio).

In [ ]:
optimizer_ft = # TODO - optymalizator
scheduler_ft = # TODO - scheduler

Uczymy sieć.

**Uwaga.** Chwila wytchnienia, chwilę to potrwa.

In [ ]:
model = train_model(model=model, criterion=criterion, optimizer=optimizer_ft, scheduler=scheduler_ft, num_epochs=5)

Ponownie weryfikujemy rezultat uczenia na zbiorze testowym z wykorzystaniem tych samych metryk:

In [ ]:
true_values, predictions = test_model(model)

print(metrics.classification_report(true_values, predictions, digits=3))

# Pytania kontrolne

W celu podsumowania wykonanych ćwiczeń proszę odpowiedź na poniższe pytania kontrolne:

1. Czy finetuning poprawił uzyskiwane rezultaty? Odpowiedź **uzasadnij**.

**Odpowiedź:**

2. Jak uzyskane wyniki plasują się na tle rezultatów uzyskanych przez autorów algorytmów SafeSO dla różnych testowanych przez nich sieci?
Przeanalizuj złożoność (liczba warst, liczba parametrów) sieci MobilNet oraz tej wykorzystanej w artykurze SafeSO i porównaj.
Weź pod uwagę liczbę parametrów oraz dokładność inferencji i rozważ przynajmniej dwie sytuacje, w których mniejsza sieć ma przewagę nad dużo większymi siecią.

**Odpowiedź:**